In [1]:
import numpy as np
import symforce
symforce.set_epsilon_to_symbol()
from symforce.values import Values
from symforce.opt.factor import Factor
import symforce.symbolic as sf


In [16]:
num_landmarks = 3
lm =np.array([
        [0, 2],
        [4, 6],
        [9, 1],
        [4,2],
        [9, 1.5],
        [9,10],
        [-5,15],
        [-7,15],])
x=np.array([[0,0],
             [1.98,.001]])
odom=np.array([[1.98,.001]])

# Maybe do range, bearing, pose, landmark
z = np.array([[2,1.57079633,0,0]])

initial_values = Values(
    poses=[sf.V2(i,j) for i,j in x],
    landmarks=[sf.V2(i,j) for i,j in lm],
    odom=odom,
    meas=z[:,0:1],
    epsilon=sf.numeric_epsilon,
)
print(initial_values)

Values(
  poses: [[0.0]
[0.0]
, [1.98]
[0.001]
],
  landmarks: [[0.0]
[2.0]
, [4.0]
[6.0]
, [9.0]
[1.0]
, [4.0]
[2.0]
, [9.0]
[1.5]
, [9.0]
[10.0]
, [-5.0]
[15.0]
, [-7.0]
[15.0]
],
  odom: [[1.98e+00 1.00e-03]],
  meas: [[2.]],
  epsilon: 2.220446049250313e-15,
)


In [17]:
def bearing_residual(
    pose: sf.Pose2, landmark: sf.V2, angle: sf.Scalar, epsilon: sf.Scalar
) -> sf.V1:
    t_body = pose.inverse() * landmark
    predicted_angle = sf.atan2(t_body[1], t_body[0], epsilon=epsilon)
    return sf.V1(sf.wrap_angle(predicted_angle - angle))

In [18]:
def range_residual(
    pose: sf.Pose2, landmark: sf.V2, angle: sf.Scalar, epsilon: sf.Scalar
) -> sf.V1:
    t_body = pose.inverse() * landmark
    predicted_angle = sf.atan2(t_body[1], t_body[0], epsilon=epsilon)
    return sf.V1(sf.wrap_angle(predicted_angle - angle))

In [19]:
def meas_residual(
    pose: sf.V2, landmark: sf.V2, meas: sf.V2, epsilon: sf.Scalar, 
) -> sf.V1:
    
    Q = np.array([[0.0,0.0],[0.0,0.0]]) 
    rng_std = .5
    bearing_std = .5
    Q[0, 0] = rng_std**2
    Q[1, 1] = bearing_std**2
    
    Q_I = np.linalg.inv(Q)
    
    rng, bearing = meas
    # predicted measurement
    dm = sf.V2(landmark - pose)
    z_pred = sf.V2([[dm.norm(epsilon=epsilon)],[sf.atan2(dm[1], dm[0],epsilon=epsilon)]])
    print(z_pred)

    # error
    e_z = meas - z_pred
    # cost
    J = e_z@Q_I@e_z.T

    return sf.V1(J)

In [20]:
def odometry_residual(
    pose_0: sf.V2, pose_1: sf.V2, odom: sf.V2, epsilon: sf.Scalar, 
) -> sf.V1:
    
    R = np.array([[0.0,0.0],[0.0,0.0]]) 
    odom_x_std = 3
    odom_y_std = 3
    R[0, 0] = odom_x_std**2
    R[1, 1] = odom_y_std**2
    R_I = np.linalg.inv(R)
    
    odom_pred = pose_1-pose_0    # error
    e_x = sf.V2(odom-odom_pred)
    # cost
    J = e_x@R_I@e_x.T
    print(J)
    return sf.V1(J)

In [21]:
from symforce.opt.factor import Factor

factors = []

# Bearing factors

for j in range(len(z)):
    factors.append(Factor(
        residual=meas_residual,
        keys=[f"poses[{z[j][2]}]", f"landmarks[{z[j][3]}]", f"meas[{j}]", "epsilon"],
    ))

# Odometry factors
for i in range(len(x) - 1):
    factors.append(Factor(
        residual=odometry_residual,
        keys=[f"poses[{i}]", f"poses[{i + 1}]", f"odom[{i}]", "epsilon"],
    ))
print(factors)

[sqrt(epsilon + (landmark0 - pose0)**2 + (landmark1 - pose1)**2)]
[atan2(landmark1 - pose1, landmark0 - pose0 + epsilon*(0.5 + sign(landmark0 - pose0)))]

0.111111111111111*(odom0 - (-pose_00 + pose_10))**2 + 0.111111111111111*(odom1 - (-pose_01 + pose_11))**2
[<symforce.opt.factor.Factor object at 0x7fbd46d46440>, <symforce.opt.factor.Factor object at 0x7fbd46d45ed0>]


In [23]:
from symforce.opt.optimizer import Optimizer

optimizer = Optimizer(
    factors=factors,
    optimized_keys=[f"pose[{i}]" for i in range(1)],
)

AssertionError: Cannot compute a linearization with respect to 0 arguments

In [79]:
result = optimizer.optimize(initial_values)

NameError: name 'optimizer' is not defined

In [80]:
from symforce.examples.robot_2d_localization.plotting import plot_solution
plot_solution(optimizer, result)

NameError: name 'optimizer' is not defined

In [ ]:
def data_association(x: np.array, z: np.array, landmarks: np.array):
    """
    Associates measurement with known landmarks using maximum likelihood
    
    @param x: state of vehicle
    @param z: measurement
    @param landmarks: map of known landmarks
    """
    dm = np.array(landmarks) - np.array(x)
    print(dm)
    rng_pred = np.linalg.norm(dm, axis=1)
    bearing_pred = np.arctan2(dm[:, 1], dm[:, 0])    
    z_error_list = np.array([rng_pred, bearing_pred]).T - np.array([z])
    # TODO: Make an actual number. Covariance of the measurement
    Q = np.array([
        [1, 0],
        [0, 1]])
    Q_I = np.linalg.inv(Q)
    J_list = []
    for z_error in z_error_list:
        J_list.append(z_error.T@Q_I@z_error)
    J_list = np.array(J_list)
    i = np.argmin(J_list)
    return i